<a href="https://colab.research.google.com/github/SolemnShark871/Churn/blob/main/SVM_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install pandas-profiling==2.7.1

## General packages
import pandas as pd
import numpy as np
import os
import math
from google.colab import drive
from pandas_profiling import ProfileReport
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import scipy
import scipy.stats as stats
from sklearn.decomposition import PCA

drive.mount('/content/drive', force_remount= True)
os.chdir('/content/drive/MyDrive/Churn') #Thomas & Favio & Juan

os.listdir("./")

Mounted at /content/drive


['personas_retiradas.csv',
 'Actividades.pdf',
 'Relational_1.pdf',
 'Actividades.gdoc',
 'Fancy Histogram of Ages.png',
 'Correlation for all features retired.png',
 'download.jpg',
 'Correlation for all features nonretired.png',
 'non_retired_people.csv',
 'retired_people.csv',
 'Datasets_Churn.ipynb',
 'Relational_1.gdoc',
 'not_scaled_not_pca_merged.csv',
 'Data_exploration.ipynb',
 'ML models.ipynb']

In [ ]:
# ML models

In [ ]:
# New cell


df_lamda = pd.read_csv('not_scaled_not_pca_merged.csv')

df_lamda.head()

,Unnamed: 0,PERSONA,AC_TP,ADM_TP,AI_TP,AP_TP,CO_TP,CU_TP,GC_TP,HD_TP,IN_TP,LI_TP,NE_TP,OP_TP,PD_TP,PM_TP,PR_TP,RE_TP,SE_TP,SM_TP,SO_TP,TD_TP,ASIGNACIONES_FACTURADAS,ASIGNACIONES_NO_FACTURADAS,AC_HR,ADM_HR,AI_HR,AP_HR,CO_HR,CU_HR,GC_HR,HD_HR,IN_HR,LI_HR,NE_HR,OP_HR,PD_HR,PM_HR,PR_HR,RE_HR,...,CU_PN,GC_PN,HD_PN,IN_PN,LI_PN,NE_PN,OP_PN,PD_PN,PM_PN,PR_PN,RE_PN,SE_PN,SM_PN,SO_PN,TD_PN,AC_RS,ADM_RS,AI_RS,AP_RS,CO_RS,CU_RS,GC_RS,HD_RS,IN_RS,LI_RS,NE_RS,OP_RS,PD_RS,PM_RS,PR_RS,RE_RS,SE_RS,SM_RS,SO_RS,TD_RS,EDAD,GENERO,TIEMPO_EMP,LAMBDA,LABEL
0,185,843,0,19,0,10,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,19,19,0.0,295.50,0.0,1656.51,41.5,0,0,0.0,0.0,0,0.0,0.00,0.0,0.00,0.0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,23,1,13,13.335903,1
1,549,1368,0,5,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,4,0.0,1012.19,0.0,0.00,0.0,0,0,0.0,0.0,0,0.0,0.00,0.0,3.83,0.0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,24,1,6,13.335903,1
2,971,21,0,29,1,10,0,0,0,0,0,0,0,0,0,5,0,0,0,25,0,0,21,13,0.0,214.25,0.0,365.49,0.0,0,0,0.0,0.0,0,0.0,0.00,0.0,50.50,0.0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1575,0,0,0,0,0,58,0,0,0,0,0,0,0,0,0,0,0,0,0,224,0,0,40,0,192,46.694458,0
3,80,571,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,271.50,0.0,113.00,292.0,0,0,0.0,0.0,0,0.0,628.00,0.0,0.00,0.0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,26,0,22,23.737822,1
4,886,2019,0,23,0,0,0,0,0,0,0,0,0,8,0,0,0,0,0,0,0,0,8,12,0.0,152.92,0.0,0.00,0.0,0,0,0.0,0.0,0,0.0,1803.51,0.0,0.00,0.0,0,...,0,0,0,0,0,0,126,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,58,0,0,0,0,0,0,0,0,27,0,13,23.737822,1
